Using Age, Experience, and Played Hours to Predict Minecraft Newsletter Subcription Status
-

**DSCI 100 010 Group 6** 

**Sayyam Arora, Michael Dickinson, Cecile Nava, Zoey Qiu** 

------------

### Introduction

The Pacific Laboratory for Artificial Intelligence (PLAI) [link their research page], a research group at UBC’s Computer Science department led by Frank Wood [link his page], is interested in understanding which characteristics and behaviours in a video game player are most predictive of game newsletter subscription for Minecraft. Group 6 poses the question of whether age (`Age`), player's experience (`experience`), and played hours (`played_hours`) are reliable predictors for subscription status (`subscribe`) in the players.csv file.  While we will be explicitly using those three mentioned variables, there are also `gender`, `name`, and `hashedEmail` which make up the 7 columns and 196 rows of `players.csv`. Below is a description of the 7 variables.  


| # | Variable         | Description                                   | Variable Type |
|---| ---------------- | ----------------------------------------------| --------------|
| 1 | experience       |  experience status of player                  | character     |
| 2 | subscribe        | whether player is subscribe to game newsletter| logical       |
| 3 | hashedEmail      | player's hashed email to identify them        | character     |
| 4 | played_hours     | how long player has played (in hours)         |        double |
| 5 | name             | name of player                                |     character |
| 6 | gender           | gender of player                              |     character |
| 7 | Age              |                                 age of player |        double |

 

Based on data exploration the mean played hours of the players is 5.8 hours, with a player reaching 223.1 hour and some not even reaching 1 full hour. The mean age of the 196 recorded players is 20, the oldest being 50 and the youngest being 8 years old.  

### METHODS USED


To explore whether a Minecraft player’s age, experience level, and hours played could predict whether they subscribe to the game’s newsletter, we used a K-Nearest Neighbors (KNN) classification model.

**Data Preparation**

We began by importing the dataset players.csv, which contains 196 observations and 7 variables, including the target variable subscribe. Only the relevant predictor variables - `age`, `played_hours`, and `experience`, were selected for analysis. We removed entries with missing values and reformatted variable types as needed, converting categorical values such as experience into an ordered factor. Irrelevant identifiers such as name and hashedEmail were excluded.

**Exploring the Data**

Before building our model, we took a closer look at the data to understand the general trends. We noticed that most players were around 20 years old, with the youngest being 8 and the oldest 50. The time spent playing Minecraft varied a lot, some players had barely played at all, while one had logged over 220 hours. This step helped us understand the range and behavior of our predictors.

**Building the Model**

To train and test our model fairly, we split the data into two parts: one for training the model and one for testing how well it performs. We used a method called cross-validation to figure out how many neighbours the KNN model should consider when making a prediction. This helped us find the best version of the model.

**Evaluating the Model**

After building the model, we tested it on the unseen portion of the data to see how accurate it was. We found that our model could correctly predict whether a player subscribed about 72.5% of the time. While the accuracy was solid, we also noticed that the model didn’t perform significantly better than chance in terms of consistency, likely because the characteristics of subscribed and unsubscribed players were quite similar.

# Discussion

<insert scatter plot with age, hours played, experience, and subscription>

To determine whether the variables Age, Hours Played, and Experience can predict Subscription status, and to visualize the relationship between the variables, we plotted the data using a scatter plot. However, the plot yielded no obvious trends among the variables—whether or not a player is subscribed to the newsletter is evenly distributed between the variables. 

To further investigate how each variable predicts Subscription status, we plotted each predictor variable individually against Subscription using histograms to visualize the distribution of players who have subscribed or not. 

##### Hours Played vs Subscription 

</ insert histogram with played hours colored by subscription> 
    
By plotting Hours Played, it can be seen that players with fewer hours played are less likely to subscribe while players with more hours played are more likely to subscribe. This is expected as it is expected that players who play the game more often would be more interested in the game and would therefore also be more inclined to subscribe to the newsletter. 

##### Age vs Subscription

<insert histogram of player age colored by subscription (without log scaling) and player subscription ration by age> 

By plotting the player’s age on a histogram, it appears that younger players are more likely to subscribe than older players. To further confirm this trend, the ratio of subscribers to non-subscribers was plotted by age. From this graph, it can be determined that younger players have a higher ratio of subscribing to the newsletter. These results also align with expectations as it is expected that younger players would be more interested in the game and therefore subscribe to the newsletter. 


##### Experience vs Subscription 

</insert histogram of player experience colored by subscription and player susbcritoin ratio by experience> 
    
By plotting the experience of players, no clear trend can be obtained. Players of all experience levels subscribed to the newsletters at a similar ratio. This is unexpected as it was expected that players with more experience would be more engaged in the game and therefore more likely to subscribe. 


##### ... not sure what to title this
Overall, by plotting each predictor individually, it can be determined that players who play more hours of the game and who are younger are more likely to subscribe to the newsletter, making the variables played_hours and age somewhat useful predictors of subscribe. Experience on the other hand, yielded no relationship with subscription suggesting that it is not a good predictor for subscription. This helps explain why our KNN classification model has an accuracy of about 72.5%—not all the predictors had strong relationships with Subscribe. 


### Implications
The implication of these results could be useful for marketing strategies. By understanding what type of players are more likely to subscribe to the newsletter, allows the content of the newsletter to be better tailored for its audience. Additionally, with this information, different marketing strategies would be employed to increase subscription rates for groups who are less likely to subscribe. 


### Future questions
- What other factors can predict whether or not a player will be subscribed to the newsletter? 
- What kind of incentives could be used to increase subscription rates? 
- … 
